# TP 1 - Parte 1 - C++

Creamos unas carpetas para organizarnos...

In [ ]:
!mkdir src
!mkdir bin

Tambien un header para mantener el codigo mas prolijo, esta estructura es la que se va a encargar de guardar y mostrar toda la informacion que almacena cada nodo.

In [ ]:
%%writefile src/node.h
#ifndef NODE_H
#define NODE_H

#include <sstream>
#include <unistd.h>

struct Node
{
  char letter;
  pid_t pid;
  pid_t parent;

  Node(char letter_)
  {
    letter = letter_;
    pid = getpid();
    parent = getppid();
  }

  std::string toString()
  {
    std::stringstream aux;
    aux << "Process: " << letter << ", PID: " << pid << ", Parent PID: " << parent;
    return aux.str();
  }
};

typedef struct Node Node;

#endif // NODE_H

Writing src/node.h


El codigo fuente, utilizamos un mapa para facilitar la creación del arbol de procesos, pasamos el tiempo que deben dormir como argumento al programa (o un tiempo default de 10 segundos) y creamos los procesos de forma recursiva.

In [ ]:
%%writefile src/process_tree.cpp
#include <cstring>
#include <iostream>
#include <map>
#include <sys/wait.h>
#include <unistd.h>
#include <vector>
#include "node.h"

#define ERROR -1
#define CHILD  0
#define INITIAL_NODE 'A'
#define DEFAULT_SLEEP_TIME 10

using namespace std;

int generateChildNodes(map<char, vector<char>> &nodes_map,
  char current_letter,
  unsigned int sleep_time);

int main(int argc, char *argv[])
{
  map<char, vector<char>> nodes_map =
  {
    {'A', {'B', 'C'}},
    {'B', {'D', 'E'}},
    {'C', {'F'}},
    {'E', {'G', 'H'}},
  };

  unsigned int sleep_time = DEFAULT_SLEEP_TIME;

  if (argc > 1)
  {
    try
    {
      sleep_time = stoi(argv[1]);
    }
    catch(const invalid_argument& e)
    {
      cerr << "Invalid time passed as parameter, using default time (" << time
      << " seconds)." << endl;
    }
  }

  return generateChildNodes(nodes_map, INITIAL_NODE, sleep_time);
}

int generateChildNodes(map<char, vector<char>> &nodes_map,
  char current_letter,
  unsigned int sleep_time)
{
  Node current = Node(current_letter);
  cout << current.toString() << endl;

  if (nodes_map.find(current.letter) == nodes_map.end())
  {
    sleep(sleep_time);
    cout << "Cerrando proceso: " << current.letter << endl;
    return EXIT_SUCCESS;
  }

  for (auto next_child: nodes_map[current.letter])
  {
    pid_t new_pid = fork();

    switch (new_pid)
    {
    case ERROR:
      cerr << "Error: " << strerror(errno) << endl;
      return EXIT_FAILURE;
      break;

    case CHILD:
      return generateChildNodes(nodes_map, next_child, sleep_time);
      break;

    default: // Parent case
      break;
    }
  }

  for (auto next_child: nodes_map[current.letter])
  {
    wait(NULL);
  }

  cout << "Cerrando proceso: " << current.letter << endl;
  return EXIT_SUCCESS;
}

Writing src/process_tree.cpp


Armamos un pequeño shell script para facilitar la corroboración del funcionamiento de nuestro programa.

In [ ]:
%%writefile run.sh
#!/usr/bin/env bash
SLEEP_TIME=5

echo "Compilando..."
g++ -std=c++17 -g src/process_tree.cpp -o bin/process_tree
if [ $? -ne 0 ]; then
    echo "Compilacion fallida."
    exit 1
fi

echo "Ejecutando..."
bin/process_tree $SLEEP_TIME 1>output 2>/dev/null &
FIRST_PROCESS=$!

sleep 1 # Esperamos un segundo a que se creen los nodos...
echo "Arbol de procesos:"
pstree -pc $FIRST_PROCESS

wait $FIRST_PROCESS # Esperamos a que se cierren...

if [ $? -eq 0 ]; then
    echo -e "\nSalida del programa:"
    cat ./output
else
    echo "Proceso termino incorrectamente."
    exit 1
fi

Writing run.sh


Le damos permisos de ejecución...

In [ ]:
!chmod +x ./run.sh

Y ejecutamos...

In [ ]:
!./run.sh

Compilando...
Ejecutando...
Arbol de procesos:
process_tree(932)─┬─process_tree(934)─┬─process_tree(936)
                  │                   └─process_tree(937)─┬─process_tree(938)
                  │                                       └─process_tree(939)
                  └─process_tree(935)───process_tree(940)

Salida del programa:
Process: A, PID: 932, Parent PID: 915
Process: B, PID: 934, Parent PID: 932
Process: E, PID: 937, Parent PID: 934
Process: D, PID: 936, Parent PID: 934
Process: C, PID: 935, Parent PID: 932
Process: H, PID: 939, Parent PID: 937
Process: G, PID: 938, Parent PID: 937
Process: F, PID: 940, Parent PID: 935
Cerrando proceso: D
Cerrando proceso: F
Cerrando proceso: H
Cerrando proceso: C
Cerrando proceso: G
Cerrando proceso: E
Cerrando proceso: B
Cerrando proceso: A
